In [31]:
rows = 'ABCDEFGHI'
cols = '123456789'

def cross(a, b):
    return [s+t for s in a for t in b]

boxes = cross(rows, cols)

row_units = [cross(r, cols) for r in rows]
column_units = [cross(rows, c) for c in cols]
square_units = [cross(rs, cs) for rs in ('ABC','DEF','GHI') for cs in ('123','456','789')]
diagonal_units1 = [[i+j for i,j in zip(rows, cols)]]
diagonal_units2 = [[i+j for i,j in zip(rows,sorted(cols,reverse=True))]]
unitlist = row_units + column_units + square_units + diagonal_units1 + diagonal_units2
units = dict((s, [u for u in unitlist if s in u]) for s in boxes)
peers = dict((s, set(sum(units[s],[]))-set([s])) for s in boxes)

def display(values):
    """
    Display the values as a 2-D grid.
    Input: The sudoku in dictionary form
    Output: None
    """
    width = 1+max(len(values[s]) for s in boxes)
    line = '+'.join(['-'*(width*3)]*3)
    for r in rows:
        print(''.join(values[r+c].center(width)+('|' if c in '36' else '')
                      for c in cols))
        if r in 'CF': print(line)
    return

def grid_values(grid):
    """
    Convert grid into a dict of {square: char} with '123456789' for empties.
    Input: A grid in string form.
    Output: A grid in dictionary form
            Keys: The boxes, e.g., 'A1'
            Values: The value in each box, e.g., '8'. If the box has no value, then the value will be '123456789'.
    """
    chars = []
    digits = '123456789'
    for c in grid:
        if c in digits:
            chars.append(c)
        if c == '.':
            chars.append(digits)
    assert len(chars) == 81
    return dict(zip(boxes, chars))
    
def eliminate(values):
    """
    Go through all the boxes, and whenever there is a box with a value, eliminate this value from the values of all its peers.
    Input: A sudoku in dictionary form.
    Output: The resulting sudoku in dictionary form.
    """
    solved_values = [box for box in values.keys() if len(values[box]) == 1]
    for box in solved_values:
        digit = values[box]
        for peer in peers[box]:
            values[peer] = values[peer].replace(digit,'')
    print(values)
    return values

def only_choice(values):
    """
    Go through all the units, and whenever there is a unit with a value that only fits in one box, assign the value to this box.
    Input: A sudoku in dictionary form.
    Output: The resulting sudoku in dictionary form.
    """
    for unit in unitlist:
        for digit in '123456789':
            dplaces = [box for box in unit if digit in values[box]]
            if len(dplaces) == 1:
                values[dplaces[0]] = digit
    print(values)
    return values

def reduce_puzzle(values):
    """
    Iterate eliminate() and only_choice(). If at some point, there is a box with no available values, return False.
    If the sudoku is solved, return the sudoku.
    If after an iteration of both functions, the sudoku remains the same, return the sudoku.
    Input: A sudoku in dictionary form.
    Output: The resulting sudoku in dictionary form.
    """
    solved_values = [box for box in values.keys() if len(values[box]) == 1]
    stalled = False
    while not stalled:
        solved_values_before = len([box for box in values.keys() if len(values[box]) == 1])
        values = eliminate(values)
        values = only_choice(values)
        solved_values_after = len([box for box in values.keys() if len(values[box]) == 1])
        stalled = solved_values_before == solved_values_after
        if len([box for box in values.keys() if len(values[box]) == 0]):
            return False
    print(values)
    return values

In [32]:
#values = grid_values('4.....8.5.3..........7......2.....6.....8.4......1.......6.3.7.5..2.....1.4......')
#values = grid_values('..3.2.6..9..3.5..1..18.64....81.29..7.......8..67.82....26.95..8..2.3..9..5.1.3..')
values = grid_values('2.............62....1....7...6..8...3...9...7...6..4...4....8....52.............3')

In [33]:
def search(values):
    print('before reducing',values)
    values = reduce_puzzle(values)
    #print('after reducing',values)
    if values is False:
        return False
    if len([box for box in values.keys() if len(values[box]) == 1]) == len(values):
        #print('found the result',values)
        return values
    s,data = min((len(values[s]), s) for s in boxes if len(values[s]) > 1)
    print('start of s and data')
    print(s)
    print(data)
    print('--end of s and data')
    for i in values[data]:
        valuescopy = values.copy()
        #print('modifying key ',data,' value from ',valuescopy[data],' to ',i)
        valuescopy[data] = i
        result = search(valuescopy)
        if result:
                return result

            

In [34]:
search(values)

before reducing {'A1': '2', 'A2': '123456789', 'A3': '123456789', 'A4': '123456789', 'A5': '123456789', 'A6': '123456789', 'A7': '123456789', 'A8': '123456789', 'A9': '123456789', 'B1': '123456789', 'B2': '123456789', 'B3': '123456789', 'B4': '123456789', 'B5': '123456789', 'B6': '6', 'B7': '2', 'B8': '123456789', 'B9': '123456789', 'C1': '123456789', 'C2': '123456789', 'C3': '1', 'C4': '123456789', 'C5': '123456789', 'C6': '123456789', 'C7': '123456789', 'C8': '7', 'C9': '123456789', 'D1': '123456789', 'D2': '123456789', 'D3': '6', 'D4': '123456789', 'D5': '123456789', 'D6': '8', 'D7': '123456789', 'D8': '123456789', 'D9': '123456789', 'E1': '3', 'E2': '123456789', 'E3': '123456789', 'E4': '123456789', 'E5': '9', 'E6': '123456789', 'E7': '123456789', 'E8': '123456789', 'E9': '7', 'F1': '123456789', 'F2': '123456789', 'F3': '123456789', 'F4': '6', 'F5': '123456789', 'F6': '123456789', 'F7': '4', 'F8': '123456789', 'F9': '123456789', 'G1': '123456789', 'G2': '4', 'G3': '123456789', 'G4'

{'A1': '2',
 'A2': '6',
 'A3': '7',
 'A4': '9',
 'A5': '4',
 'A6': '5',
 'A7': '3',
 'A8': '8',
 'A9': '1',
 'B1': '8',
 'B2': '5',
 'B3': '3',
 'B4': '7',
 'B5': '1',
 'B6': '6',
 'B7': '2',
 'B8': '4',
 'B9': '9',
 'C1': '4',
 'C2': '9',
 'C3': '1',
 'C4': '8',
 'C5': '2',
 'C6': '3',
 'C7': '5',
 'C8': '7',
 'C9': '6',
 'D1': '5',
 'D2': '7',
 'D3': '6',
 'D4': '4',
 'D5': '3',
 'D6': '8',
 'D7': '1',
 'D8': '9',
 'D9': '2',
 'E1': '3',
 'E2': '8',
 'E3': '4',
 'E4': '1',
 'E5': '9',
 'E6': '2',
 'E7': '6',
 'E8': '5',
 'E9': '7',
 'F1': '1',
 'F2': '2',
 'F3': '9',
 'F4': '6',
 'F5': '5',
 'F6': '7',
 'F7': '4',
 'F8': '3',
 'F9': '8',
 'G1': '6',
 'G2': '4',
 'G3': '2',
 'G4': '3',
 'G5': '7',
 'G6': '9',
 'G7': '8',
 'G8': '1',
 'G9': '5',
 'H1': '9',
 'H2': '3',
 'H3': '5',
 'H4': '2',
 'H5': '8',
 'H6': '1',
 'H7': '7',
 'H8': '6',
 'H9': '4',
 'I1': '7',
 'I2': '1',
 'I3': '8',
 'I4': '5',
 'I5': '6',
 'I6': '4',
 'I7': '9',
 'I8': '2',
 'I9': '3'}

In [16]:
unitlist

[['A1', 'A2', 'A3', 'A4', 'A5', 'A6', 'A7', 'A8', 'A9'],
 ['B1', 'B2', 'B3', 'B4', 'B5', 'B6', 'B7', 'B8', 'B9'],
 ['C1', 'C2', 'C3', 'C4', 'C5', 'C6', 'C7', 'C8', 'C9'],
 ['D1', 'D2', 'D3', 'D4', 'D5', 'D6', 'D7', 'D8', 'D9'],
 ['E1', 'E2', 'E3', 'E4', 'E5', 'E6', 'E7', 'E8', 'E9'],
 ['F1', 'F2', 'F3', 'F4', 'F5', 'F6', 'F7', 'F8', 'F9'],
 ['G1', 'G2', 'G3', 'G4', 'G5', 'G6', 'G7', 'G8', 'G9'],
 ['H1', 'H2', 'H3', 'H4', 'H5', 'H6', 'H7', 'H8', 'H9'],
 ['I1', 'I2', 'I3', 'I4', 'I5', 'I6', 'I7', 'I8', 'I9'],
 ['A1', 'B1', 'C1', 'D1', 'E1', 'F1', 'G1', 'H1', 'I1'],
 ['A2', 'B2', 'C2', 'D2', 'E2', 'F2', 'G2', 'H2', 'I2'],
 ['A3', 'B3', 'C3', 'D3', 'E3', 'F3', 'G3', 'H3', 'I3'],
 ['A4', 'B4', 'C4', 'D4', 'E4', 'F4', 'G4', 'H4', 'I4'],
 ['A5', 'B5', 'C5', 'D5', 'E5', 'F5', 'G5', 'H5', 'I5'],
 ['A6', 'B6', 'C6', 'D6', 'E6', 'F6', 'G6', 'H6', 'I6'],
 ['A7', 'B7', 'C7', 'D7', 'E7', 'F7', 'G7', 'H7', 'I7'],
 ['A8', 'B8', 'C8', 'D8', 'E8', 'F8', 'G8', 'H8', 'I8'],
 ['A9', 'B9', 'C9', 'D9', 'E9',

In [37]:
values = {'A1': '2', 'A2': '356789', 'A3': '34789', 'A4': '1345789', 'A5': '134578', 'A6': '134579', 'A7': '13569', 'A8': '1345689', 'A9': '145', 'B1': '45789', 'B2': '57', 'B3': '34789', 'B4': '1345789', 'B5': '134578', 'B6': '6', 'B7': '2', 'B8': '1345', 'B9': '14589', 'C1': '45689', 'C2': '35689', 'C3': '1', 'C4': '34589', 'C5': '23458', 'C6': '23459', 'C7': '35', 'C8': '7', 'C9': '45689', 'D1': '14579', 'D2': '12579', 'D3': '6', 'D4': '457', 'D5': '123457', 'D6': '8', 'D7': '1359', 'D8': '12359', 'D9': '1259', 'E1': '3', 'E2': '1258', 'E3': '248', 'E4': '145', 'E5': '9', 'E6': '1245', 'E7': '156', 'E8': '12568', 'E9': '7', 'F1': '15789', 'F2': '125789', 'F3': '2789', 'F4': '6', 'F5': '12357', 'F6': '57', 'F7': '4', 'F8': '123589', 'F9': '12589', 'G1': '1679', 'G2': '4', 'G3': '237', 'G4': '13579', 'G5': '13567', 'G6': '13579', 'G7': '8', 'G8': '12569', 'G9': '12569', 'H1': '16789', 'H2': '137', 'H3': '5', 'H4': '2', 'H5': '134678', 'H6': '13479', 'H7': '1679', 'H8': '46', 'H9': '1469', 'I1': '17', 'I2': '126789', 'I3': '2789', 'I4': '145789', 'I5': '145678', 'I6': '14579', 'I7': '15679', 'I8': '124569', 'I9': '3'}

In [50]:
dict(zip([(values[box],box) for box in values.keys() if len(values[box]) == 2]))

ValueError: dictionary update sequence element #0 has length 1; 2 is required

In [11]:
dict((s, [u for u in unitlist if s in u]) for s in boxes)

In [12]:
t.values


AttributeError: 'zip' object has no attribute 'values'